<h1 style="text-align: center;">Forecasting traffic in Paris</h1>


In [ ]:
import numpy as np
import pandas as pd
from prophet import Prophet
import sys

sys.path.append('/Users/clovispiedallu/Library/Mobile Documents/com~apple~CloudDocs/Documents/Machine Learning/Challenge/BCG/Datathon_BCG_colissiMONSTRE/datathon_bcg/module')

from dataprep import *

%load_ext autoreload
%autoreload 2

In [ ]:
df_convention_raw = load_traffic_data(arc='convention')
df_champs_raw = load_traffic_data(arc='champs')
df_sts_raw = load_traffic_data(arc='sts')

df_convention_clean = traiter_donnees(df=df_convention_raw, arc='convention')
df_champs_clean = traiter_donnees(df_champs_raw, arc='champs')
df_sts_clean = traiter_donnees(df_sts_raw, arc='sts')